# Scraping company numbers from names

This notebook details the process of fetching company numbers for a list of company names. 

The list of company names comes from MyEU. These have been filtered to those with "Limited" or "ltd" in the name.

An early test suggests that even those with variations, such as "A1 Arborists LTD/Limited" still result in the right match first time, but we will build in some data validation.

## Upload company names

First we upload a file to this runtime - this has to be done each time you run this notebook. ltdcompaniesgettingfunding.csv

In [7]:
#import the pandas library
import pandas as pd
#import the csv using a pandas function
ltdcompaniesgettingfunding = pd.read_csv("ltdcompaniesgettingfunding.csv")
#Show the first few rows
pd.DataFrame.head(ltdcompaniesgettingfunding)

,Row Labels,Ltd or limited
0,A1 Arborists LTD/Limited,2
1,Ensilica Limited Ltd,2
2,Qoob Limited Ltd,2
3,Sei Oxford Office Limited*stockholmenvironment...,2
4,0 Infinity Limited,1


We just want to loop through one column and generate some URLs.

A search URL looks like this: 

`https://beta.companieshouse.gov.uk/search?q=A1+Arborists+LTD%2FLimited`

The search term is at the end, with spaces replaced by the `+` operator.

In [110]:
# -*- coding: utf-8 -*-
#Specify the base of the search URL
searchurlbase = "https://beta.companieshouse.gov.uk/search?q="
#How many items do we have to loop through?
print(len(ltdcompaniesgettingfunding['Row Labels']))
#Test using .replace() to replace spaces and slashes
testtxt = "A1 Arborists LTD/Limited"
print(searchurlbase+testtxt.replace(" ","+").replace("/","%2F"))

#import regex
# taken from https://stackoverflow.com/questions/1342000/how-to-make-the-python-interpreter-correctly-handle-non-ascii-characters-in-stri
import re
#Create regex to replace odd characters
strip_unicode = re.compile("([^-_a-zA-Z0-9!@#%&=,/'\";:~`\$\^\*\(\)\+\[\]\.\{\}\|\?\<\>\\]+|[^\s]+)")

#Now loop through the column and generate the URLs
#First create an empty list
urllist = []
#Now loop through the other list
for i in ltdcompaniesgettingfunding['Row Labels']:
  #Create a search URL and add it to the list
  #print(i)
  #Correct odd characters
  i = strip_unicode.sub('',i)
  #print(i)
  fullurl = searchurlbase+i.replace(" ","+").replace("/","%2F")
  urllist.append(fullurl)

#Check how long our new list is now
print(len(urllist))

Streaming output truncated to the last 5000 lines.
Idea - M Ltd.
Idea - M Ltd.
Ideal Boilers Limited
Ideal Boilers Limited
Ideas Forward Ltd
Ideas Forward Ltd
Ids Ingegneria Dei Sistemi (Uk) Limited
Ids Ingegneria Dei Sistemi (Uk) Limited
Idus Consultancy Ltd
Idus Consultancy Ltd
Ige Energy Services (Uk) Limited
Ige Energy Services (Uk) Limited
Ignitas Limited
Ignitas Limited
Igs Strategic Communications Limited
Igs Strategic Communications Limited
iGuzzini illuminazione UK Ltd
iGuzzini illuminazione UK Ltd
IKEA Ltd
IKEA Ltd
IKEA Ltd.
IKEA Ltd.
Ikinema Limited
Ikinema Limited
IKKAIDO LTD
IKKAIDO LTD
Ilbey Training International Ltd.
Ilbey Training International Ltd.
Illumina Cambridge Ltd
Illumina Cambridge Ltd
Illuminations (Television) Limited
Illuminations (Television) Limited
Ilmor Engineering Limited
Ilmor Engineering Limited
IM Travel Limited
IM Travel Limited
Image Analysis Limited
Image Analysis Limited
Image on Glass Limited
Image on Glass Limited
Image Processing Techniques L

## Export the search URLs

To prevent us having to run the code above again, we export the data and then download it from the file section on the left.

In [0]:
#Add it to our dataframe
ltdcompaniesgettingfunding['searchurl'] = urllist
#Check
pd.DataFrame.head(ltdcompaniesgettingfunding)
#Export
ltdcompaniesgettingfunding.to_csv("ltdcompanysearch.csv")

## Now for the scraper

We need to scrape those URLs, so let's install the library we need.

In [19]:
#install scraperwiki
!pip install scraperwiki
import scraperwiki

     |████████████████████████████████| 1.1MB 3.9MB/s 
     |████████████████████████████████| 471kB 47.6MB/s 
  Created wheel for scraperwiki: filename=scraperwiki-0.5.1-cp36-none-any.whl size=6547 sha256=4c03bd8395897c1d5276a4c1159a44c087992d3c9a02b2cfcd63e1989d645287
  Stored in directory: /root/.cache/pip/wheels/6a/6e/60/e13b585339206922e816bb90c355b79aa077ab2b15d7cc26a7
  Created wheel for alembic: filename=alembic-1.4.0-py2.py3-none-any.whl size=157561 sha256=c24bcdce40d52202eaf6b3f47f3dc944bf130ea1245aa1e1028701affcda9ee0
  Stored in directory: /root/.cache/pip/wheels/26/5e/30/84097f2b9e8215a03af766c647ac6dbef8f2f522af31e8c192
  Created wheel for Mako: filename=Mako-1.1.1-cp36-none-any.whl size=75410 sha256=ffde681f39007d8fe1d1572810f173511469400109ae2d109563d4b36cca3437
  Stored in directory: /root/.cache/pip/wheels/ff/50/a9/0ddeed8679a1fb65bf4677cb9c92701828b2c1821e22ef72fd
Successfully built scraperwiki alembic Mako


And some others.

In [20]:
import lxml.html
import re
!pip install cssselect
import cssselect

Now

In [0]:
#Create empty dict to store results
record = {}
#Create a function to scrape the appointments page
def scrapefirstresult(url):
    print("scraping",url)
    html = scraperwiki.scrape(url)
    root = lxml.html.fromstring(html) # turn our HTML into an lxml object
    #There are multiple results
    links = root.cssselect('h3 a')
    #print(len(links))
    #print(links)
    #We just want the first result - and the link
    if len(links) > 0:
      link = links[0].attrib['href']
      #Some searches have 0 results
      #Grab the text too, with the name
      companyname = links[0].text_content()
      #Replace part of the URL to just get the company number
      record['companyno'] = link.replace("/company/","")
      record['companyname'] = companyname.rstrip().replace("\n                                        ","")
      record['searchurl'] = url
      #print(record)
      scraperwiki.sql.save(['companyno'], record, table_name = 'companynumbers')

Let's check we still have 4,400 URLs:

In [113]:
print(len(ltdcompaniesgettingfunding['searchurl']))

4426


And loop through those:

In [0]:
for i in ltdcompaniesgettingfunding['searchurl']:
  scrapefirstresult(i)

In [123]:
allscrapeddata = scraperwiki.sql.select("* from companynumbers")
print(len(allscrapeddata))
#Previously it only went to 464 because of a problem with a non-ASCII character, so we checked the last entry before that
#Then 1404 was a no result
print(ltdcompaniesgettingfunding['searchurl'][1404])

4317
https://beta.companieshouse.gov.uk/search?q=Excel+English+Language+School+Ltd


## Export the scraped data

Now we export it.

In [124]:
#Create a dataframe and assign the data to it
allscrapeddatadf = pd.DataFrame(allscrapeddata)
#Show how many rows
print(len(allscrapeddatadf))
#Print the first few rows (head)
print(allscrapeddatadf.head())
allscrapeddatadf.to_csv('allscrapeddata.csv')

4317
  companyno  ...                companyname
0  06449984  ...     'A1' ARBORISTS LIMITED
1  04220106  ...           ENSILICA LIMITED
2  09150745  ...          QOOB LIMITED LTD.
3  04404220  ...  SEI OXFORD OFFICE LIMITED
4  10911227  ...         0 INFINITY LIMITED

[5 rows x 3 columns]


## Scrape the company URLs

Now we have a list of company numbers, we can generate URLs for each of those and scrape details on each company.

In [0]:
#Create empty dict to store results
record = {}
#Create a function to scrape the appointments page
def scrapecompanypage(url):
    #print("scraping",url)
    html = scraperwiki.scrape(url)
    root = lxml.html.fromstring(html) # turn our HTML into an lxml object
    #There are multiple results
    #If there are multiple ones they all start with span id="sic and then a digit
    #So we do a partial match
    #See https://www.antimath.info/css/css-target-elements-by-partial-class-or-id-name/
    sics = root.cssselect('li span[id^="sic"]')
    print(len(sics))
    #Loop through the SIC codes - we want one on each row of our data so they can be pivoted
    for sic in range(0,len(sics)):
      #Store the full text
      fulltext = sics[sic].text_content()
      #Split on the hyphen so we can extract the code separately
      splitcode = fulltext.split(" - ")
      siccode = splitcode[0]
      sictext = splitcode[1]
      record['sic'] = siccode.replace("\n                    ","").rstrip()
      record['sictext'] = sictext.rstrip().replace("\n                    ","")
      record['sicfulltext'] = fulltext.rstrip().replace("\n                    ","")
      record['url'] = url
      record['companynumber'] = url.replace("https://beta.companieshouse.gov.uk/company/","")
      #Generate unique id for primary key
      record['uniqueid'] = url+siccode
      #Change name to companysicstest while testing
      scraperwiki.sql.save(['uniqueid'], record, table_name = 'companysicsreal')

#Test out on a page we know has multiple SIC codes
##testurl = "https://beta.companieshouse.gov.uk/company/10425994"
#scrapecompanypage(testurl)

In [66]:
#This only works when the above code is changed for testing
companysics = scraperwiki.sql.select("* from companysicstest")
#Create a dataframe and assign the data to it
companysicsdf = pd.DataFrame(companysics)
#Show how many rows
print(len(companysicsdf))
#Print the first few rows (head)
print(companysicsdf)


2
     sic  ...                                           uniqueid
0  56102  ...  https://beta.companieshouse.gov.uk/company/104...
1  56103  ...  https://beta.companieshouse.gov.uk/company/104...

[2 rows x 6 columns]


In [76]:
test = "/officers"
print(test[:9])

/officers


In [126]:
#Loop through those
for i in allscrapeddatadf['companyno']:
  print(i)
  #Some are officer URLs not companies
  #We need to fix the scraper
  if i[:9] != "/officers":
    companyurl = "https://beta.companieshouse.gov.uk/company/"+i
    scrapecompanypage(companyurl)

Streaming output truncated to the last 5000 lines.
08267469
1
10488780
1
01326526
1
02242542
2
08219154
1
05363294
1
00715643
1
08820146
1
02895026
1
12242984
1
SC174922
1
10299189
1
01237753
1
05736799
1
03931606
1
08057353
1
02562099
1
02590031
1
02396114
2
12370885
1
09565046
1
04052815
1
03639438
1
06217134
1
04021002
2
11680078
1
07921001
1
NI001006
1
04032392
2
05045547
1
03682759
1
01923041
1
06270723
1
08334706
1
SC080870
1
05018842
1
03242704
2
01955749
1
09590203
1
05148143
1
NI048778
1
04860134
1
06698161
1
07096894
3
02143366
1
04454812
1
04035328
1
04335734
2
02926939
1
07788776
1
07496371
1
SC345934
1
07208355
3
03154411
4
00741598
1
10614737
1
03024799
1
03900983
2
08004482
2
06936571
2
05068195
1
03295452
1
02721051
1
05794136
4
10282894
1
09825182
2
00322137
1
08287526
2
04355374
4
05716133
1
03499114
1
05940529
1
07171556
1
02391370
1
01986283
2
05983680
1
08654724
1
06666884
1
03625145
1
01613547
2
05475534
1
07107317
2
06286687
1
SC235628
1
03564291
1
03480995
1
060

Now to export

In [127]:
companysicsreal2 = scraperwiki.sql.select("* from companysicsreal")
print(companysicsreal2)
#Create a dataframe and assign the data to it
companysicsrealdf = pd.DataFrame(companysicsreal2)
#Show how many rows
print(len(companysicsrealdf))
#Print the first few rows (head)
print(companysicsrealdf.head())
companysicsrealdf.to_csv('companysicsrealdf.csv')

[{'sic': '02100', 'sictext': 'Silviculture and other forestry activities', 'sicfulltext': '02100 - Silviculture and other forestry activities', 'url': 'https://beta.companieshouse.gov.uk/company/06449984', 'companynumber': '06449984', 'uniqueid': 'https://beta.companieshouse.gov.uk/company/06449984\n                    02100'}, {'sic': '62090', 'sictext': 'Other information technology service activities', 'sicfulltext': '62090 - Other information technology service activities', 'url': 'https://beta.companieshouse.gov.uk/company/04220106', 'companynumber': '04220106', 'uniqueid': 'https://beta.companieshouse.gov.uk/company/04220106\n                    62090'}, {'sic': '62012', 'sictext': 'Business and domestic software development', 'sicfulltext': '62012 - Business and domestic software development', 'url': 'https://beta.companieshouse.gov.uk/company/09150745', 'companynumber': '09150745', 'uniqueid': 'https://beta.companieshouse.gov.uk/company/09150745\n                    62012'}, {'